In [3]:
#imports here

import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import os
import re
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
import pickle

### This is where the fun begins

In [10]:
#create a function to change the date column into a datetime series
MONTH_TO_NUM = {"JAN" : "01", "FEV" : "02", "MAR" : "03", "AVR" : "04", "MAI" : "05", "JUN" : "06", "JUI" : "07", "AOU" : "08", "SEP" : "09", "OCT" : "10", "NOV" : "11", "DEC" : "12"}

def create_datetime(df, file_name):
    year= file_name[:4]

    #error handeling on invalid days anything greater than 31 (should add a clause to check per month type on year type as well):
    mask = pd.to_numeric(df["date"].str[:2]) <= 31
    df = df[mask]

    df["date"] = pd.to_datetime(year + df["date"].str[-3:].map(MONTH_TO_NUM) + df["date"].str[:2])
    if(file_name[5:7] == "01"):
        #mask used to 
        mask = df["date"].dt.month == 12
        df["date"][mask] -= pd.DateOffset(years = 1)
    return df

In [ ]:
#initalize all values here:
#capitals as per naming convention for python when you want to assign a constant
#padding needed to not read repetative noise
OFFSET = 6
#height in pixels of each row
HEIGHT = 27
#config for the ocr config arg
CONFIG = "--psm 12 --oem 1"
#collect the year for the files later by regular expression but for the time being, since we already know that the numbers are in the range of 0-9 position, we'll just slice

#set up paths
SOURCE_PATH = "data/unprocessed/debit/"
SAVE_PATH = "data/processed/debit/"
#-1 to remove the "/"
file_names = os.listdir(SOURCE_PATH)
for file in file_names:
    #create vars to hold each column (we will zip em together later to turn into a dataframe)
    #also reset values
    date = []
    description = []
    charge =[]
    total = []

    #here we will assume that the pdf size is 1700 to 2200 (later we will return to add more adaptability)
    #set up base crop dimensision
    left = 325
    top = 397
    right = 1650
    #bottom will be under constant change per loop, so we'll just initalize it
    bottom = top + HEIGHT

    #come back later to allow for more than one page to be processed
    image = convert_from_path(SOURCE_PATH + file)[1]

    #make shift do-while loop
    #we shall read all the way to the bottom, cleaning will be done in the next steps
    #we could switch it for a forloop sure by dividing the remainging length by the total rows height and just seeing how many times we can run it, but I already wrote this
    while bottom <= 2200:
        #Crop the image to just read row by row
        seg = image.crop((left, top, right, bottom))

        #Lets OCR this thing
        info = pytesseract.image_to_string(seg, config=CONFIG)
        print(info, file)
        info = info.rstrip("\n")
        info = info.split("\n\n")

        #some in precosion error handeling
        if len(info) > 3:
            date.append(info[0])
            description.append(info[1])
            charge.append(info[2])
            total.append(info[3])

        #reset the crop values
        top = bottom + OFFSET
        bottom = top + HEIGHT
    
    #turn into a df
    file_name = file[:10]
    df = pd.DataFrame(list(zip(date, description, charge, total)), columns=["date", "description", "charge", "total"])

    #here we clean the data in all the csvs

    #create a mask to filter out invalid data based on invalid dates (potentially remove and turn into a method to be used in other functions)
    mask = df["date"].str.contains(r"[0-3][0-9]\s*[A-Z]{3}")
    #remove exess
    df= df[mask]
    #To make sure we only have the formate we wish to use
    df['date'] = df["date"].str.extract(r"([0-3][0-9]\s*[A-Z]{3})", expand=False)

    #change the format of total and charge so we can use it without error later
    df["total"] = df["total"].replace(r"[^-*\d]", "", regex=True)
    df["charge"] = df["charge"].replace(r"[^\d]", "", regex=True)
    #turn it into a decimal
    df["total"] = df["total"].str[:-2] + "." + df["total"].str[-2:]
    df["charge"] = df["charge"].str[:-2] + "." + df["charge"].str[-2:]

    #add datetime:
    df = create_datetime(df,file)
    #index set to false as it is not needed
    df.to_csv(SAVE_PATH + file_name + ".csv", index=False)




To note over five years of data, only three anomolies showed up and one page where a second page was given to store more transaction information <br>
solution for anomoly: all three had the same 

In [14]:
#here we will put all the csvs into one big file to play around with
CSV_PATH = "data/processed/debit"
files = os.listdir(CSV_PATH)
#read all files as csvs and put into a list
files_to_concat = [pd.read_csv(CSV_PATH+'/'+file) for file in files]


main_df = pd.concat(files_to_concat, ignore_index=True)

expenses = main_df[main_df["total"].diff().fillna(0) <0]
#export all transactions
main_df.to_csv("main.csv", index=False)
#exports all expenses
expenses.to_csv("expenses.csv", index=False)


,date,description,charge,total
1,2016-11-01,FRAIS FIXES,0.44,-2.94
2,2016-12-01,FR TENUE DE CPTE,2.50,-5.44
3,2016-12-01,FRAIS FIXES,2.75,-8.19
4,2016-12-01,INTERETS DECOUVERT,0.05,-8.24
6,2016-12-30,FRAIS FIXES,2.75,9.01
...,...,...,...,...
1387,2022-03-18,COMPTE DIVERS PAYPAL,24.58,1828.76
1388,2022-03-22,VIREMENT INTERAC,96.14,1732.62
1390,2022-03-28,MOBILE VIREMENTMC,728.92,1103.70
1391,2022-03-29,COMPTE DIVERS PAYPAL,44.43,1059.27


Due to time restraints I had to fix some files by hand so that the values I'd use for the model would be at their most accurate

In [3]:
#catagory creations, key words that will determine where an expense belongs
bills = [
    "frais",
    "fr tenue",
    "interets decouvert",
    "ASSURANCE SUN LIFE",
    "TRANS. EXCEDENT."
]
mobile_payments = [
    "mobile",
    "internet paiem"
]
transport = [
    "billetterie lon",
    "billeterie du",
    "stm",
    "BILLETTERIE TER"
]
resturants = [
    "la belle provin",
    "mcdonald",
    "krispy kreme",
    "tim horton",
    "tims hortons",
    "cafe gar",
    "subway",
    "cantine",
    "restaurants",
    "dairy queen",
    "dunns",
    "eggsquis",
    "thai express",
    "pizzahut",
    "n g drive in",
    "pizza pizza",
    "amir",
    "frite alors",
    "au bon sushi",
    "ROTISSERIE",
    "boustan",
    "TUTTI FRUTTI",
    "benny bbq",
    "nos thes",
    "wendys",
    "thai expr",
    "buffet",
    "EAST SIDE MARIO",
    "kalbi",
    "a&w",
    "DISTILLERIE",
    "KINTON RAMEN",
    "resto",
    "harvey",
    "cafe",
    "LA DIPERIE"
]
groceries = [
    "maxi",
    "super c",
    "iga extra",
    "depanneur",
    "provigo",
    "ALIMENTS KIM PH",
    "FRUITERIE 440",
    "metro"
]
gas_station = [
    "petro-canada",
    "couche-tard",
    "couchetard",
    "ULTRAMAR"
]
withdraws = [
    "retrait"
]
food = [
    "LE TERROIR DE D"
]
office_ware = [
    "bureau en gros",
    "BKSTR CHAMPLAIN"
]
hard_ware = [
    "home depot",
    "canadian tire"
]
home_ware = [
    "economax"
]
electronics = [
    "best buy",
    "canada computer",
    "eb games",
    "superclub video"
]
retail = [
    "wal-mart",
    "tigre geant",
    "dollarama",
    "animal expert",
    "party expert",
    "tristan",
    "marche",
    "IMAGINUS CANADA", # posters
    "jeans factory",
    "game keeper", #board game store,
    "aldo",
    "yellow",
    "AUBAINERIE",
    "CLAIR DE LUNE",
    "lush",
    "giant tiger",
    "bongia",
    "sephora",
    "FERNANDE FLEURI",
    "SQDC",
    "SAQ",
    "ardene",
    "MEGA VENTE",
    "LA VIE EN ROSE",
    "h&m"
]
medical = [
    "pharmaprix",
    "jean coutu",
    "CLIN VISUELLE",
    "pharm"
]
paypal = [
    "paypal"
]
interact_transfers = [ 
    "interac"
]
investments = [ 
    "invest"
]
activity = [ 
    "BREAKOUT",
    "DOMAINE DU SKI",
    "MOUNTAIN EQUIPM"
]
cheque = [
    "cheque"
]

In [19]:
#here we will do some catagorizing of the descriptions on the data and create a make shift catagorizer
data = pd.read_csv("expenses.csv").dropna()
#remove total as it will no longer be needed
data.pop("total")
#create new column to be modified
data["catagory"] = pd.NA

#mask creations to add catagory type
data.loc[data["description"].str.contains("|".join(bills), case=False), "catagory"] = "bills" # bills
data.loc[data["description"].str.contains("|".join(mobile_payments), case=False),'catagory'] = "mobile_payments" #bills
data.loc[data["description"].str.contains("|".join(transport), case=False),'catagory'] = "transport" #bills
data.loc[data["description"].str.contains("|".join(resturants), case=False),'catagory'] = "resturants" #food
data.loc[data["description"].str.contains("|".join(groceries), case=False),'catagory'] = "groceries" #food
data.loc[data["description"].str.contains("|".join(gas_station), case=False),'catagory'] = "gas_station" #bills/leisure
data.loc[data["description"].str.contains("|".join(withdraws), case=False),'catagory'] = "withdraws" #other
data.loc[data["description"].str.contains("|".join(food), case=False),'catagory'] = "food" # food/retail
data.loc[data["description"].str.contains("|".join(office_ware), case=False),'catagory'] = "office_ware" #retail
data.loc[data["description"].str.contains("|".join(hard_ware), case=False),'catagory'] = "hard_ware" #retail
data.loc[data["description"].str.contains("|".join(home_ware), case=False),'catagory'] = "home_ware" #retail
data.loc[data["description"].str.contains("|".join(electronics), case=False),'catagory'] = "electronics" #retail
data.loc[data["description"].str.contains("|".join(retail), case=False),'catagory'] = "retail" #retail
data.loc[data["description"].str.contains("|".join(medical), case=False),'catagory'] = "medical" #bills
data.loc[data["description"].str.contains("|".join(paypal), case=False),'catagory'] = "paypal" #paypal
data.loc[data["description"].str.contains("|".join(interact_transfers), case=False),'catagory'] = "interact" #interact
data.loc[data["description"].str.contains("|".join(investments), case=False),'catagory'] = "investment" #investment
data.loc[data["description"].str.contains("|".join(activity), case=False),'catagory'] = "activity" #luxury
data.loc[data["description"].str.contains("|".join(cheque), case=False),'catagory'] = "cheque" #cheques
data.loc[data.catagory.isna(), 'catagory'] = "other" #for uncatagorized

#no longer needed past this point
data.pop("description")

#group all catagory into bigger groups
bills_cat = ["bills", "mobile_payments", "medical"]
food_cat = ["resturants", "groceries", "food"]
travel_cat = ["transport", "gas_station"]
third_cat = ["paypal", "interact"]
retail_cat = ["retail", "hard_ware", "home_ware", "electronics"]
sum_list = ["bills", "food", "travel", "third_party", "retail"]
data.loc[data.catagory.isin(bills_cat), "catagory"] = "bills"
data.loc[data.catagory.isin(food_cat), "catagory"] = "food"
data.loc[data.catagory.isin(travel_cat), "catagory"] = "travel"
data.loc[data.catagory.isin(third_cat), "catagory"] = "third_party"
data.loc[data.catagory.isin(retail_cat), "catagory"] = "retail"
data.loc[~data.catagory.isin(sum_list), "catagory"] = "other"
#to confirm group sizes
data.catagory.unique()

array(['bills', 'travel', 'food', 'other', 'retail', 'third_party'],
      dtype=object)

In [20]:
#calculate total spent per month
data.date = data.date.astype("datetime64")
#to remove the day
data.date = data.date.dt.strftime("%Y-%m")
total_per_catagory = data.groupby(["date", "catagory"]).sum().reset_index()
total = total_per_catagory.groupby("date").sum()
total.columns = ["total"]
data = pd.merge(total_per_catagory, total, on="date", how="left")
data["percentage"] = data.charge/data.total
data = data.drop(["charge", "total"], axis = 1)
data

,date,catagory,percentage
0,2016-11,bills,1.000000
1,2016-12,bills,1.000000
2,2017-02,bills,0.868819
3,2017-02,food,0.081687
4,2017-02,travel,0.049494
...,...,...,...
268,2022-02,bills,0.190790
269,2022-02,third_party,0.809210
270,2022-03,bills,0.450229
271,2022-03,third_party,0.549771


In [21]:
#to fill in missing values

#to grab all avaliable dates
date_df = pd.DataFrame(data.date.unique(), columns=["date"])

#to grap all avaliable catagories
catagory_df = pd.DataFrame(data.catagory.unique(), columns = ["catagory"])
final_df = date_df.merge(catagory_df, how = "cross").merge(data, how = "left", on = ["date", "catagory"]).fillna(0.0)
final_df.to_csv("catagory_data.csv", index = False)

In [18]:
final_df = pd.read_csv("catagory_data.csv")
final_df.date = final_df.date.astype("datetime64")
final_df.set_index("date", inplace=True)

In [5]:
#Building best model
#I expect this method to run for a while
def get_best_model(df, p_s:list, d_s:list, q_s:list):
    best_model = None
    best_BIC = None
    #Lets do this triple nested loop
    for p in p_s:
        for d in d_s:
            for q in q_s:
                mod = ARIMA(df, order=(p,d,q)).fit()
                #create a base case
                if not best_model:
                    best_model = mod
                    best_BIC = best_model.bic
                #this is where the fine tuning happens
                if best_BIC > mod.bic:
                    best_model = mod
                    best_BIC = mod.bic
    
    return best_model



In [17]:
#create model for each unique values
for c_tag in list(final_df.catagory.unique()):
    data = final_df.loc[final_df.catagory == c_tag, "percentage"]["2017":]
    print(c_tag)
    train_index = int(len(data) * 0.8) + 1
    data = data.iloc[:train_index]
    mod = get_best_model(data, list(range(0,12)), [0,1], list(range(0,12)))
    print("completed")
    file = open(f"models/{c_tag}.md", "xb")
    pickle.dump(mod, file)
    file.close()


bills


c:\Users\jibz1\school\habit_spender_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\jibz1\school\habit_spender_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\jibz1\school\habit_spender_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\jibz1\school\habit_spender_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\jibz1\school\habit_spender_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No fr

completed
food


c:\Users\jibz1\school\habit_spender_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\jibz1\school\habit_spender_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\jibz1\school\habit_spender_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\jibz1\school\habit_spender_env\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\jibz1\school\habit_spender_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: 

completed
travel


...

completed
retail


c:\Users\jibz1\school\habit_spender_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\jibz1\school\habit_spender_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\jibz1\school\habit_spender_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\jibz1\school\habit_spender_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\jibz1\school\habit_spender_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No fr

completed
third_party


c:\Users\jibz1\school\habit_spender_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\jibz1\school\habit_spender_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\jibz1\school\habit_spender_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\jibz1\school\habit_spender_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\jibz1\school\habit_spender_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No fr

completed


all catagories:  
bills  
food  
travel  
other  
retail  
third_party  

If time left, for the arimas, do one more monthly calculation but on the differences